## Seção 7: Machine learning com Spark

In [1]:
import os
import pyspark
import findspark
from pyspark.sql import SparkSession

os.environ["SPARK_HOME"] = "C:\\ApacheSpark\\spark-3.5.3-bin-hadoop3"

findspark.init()
spark = SparkSession.builder.master('local[*]').getOrCreate()

### 55. Fundamentos de Machine Learning

#### Oque é machine learning?

- Machine Learning (ML) é um subcampo da inteligência artificial (IA) que permite que sistemas aprendam e melhorem com a experiência sem serem explicitamente programados para cada tarefa. Em vez disso, eles usam dados para treinar modelos que identificam padrões e fazem previsões ou decisões. O aprendizado ocorre por meio de algoritmos que ajustam as previsões com base nos dados de entrada e nos resultados esperados.

#### Como funciona
O ML se baseia em algoritmos que processam grandes volumes de dados para descobrir padrões. Existem diferentes tipos de aprendizado:

- Supervisionado: O modelo é treinado com dados rotulados (exemplo: entrada e resposta esperada). Útil para classificação e regressão.
- Não supervisionado: Trabalha com dados não rotulados, encontrando padrões e grupos ocultos nos dados. Aplicável para segmentação e redução de dimensionalidade.
- Reforço: O modelo aprende por tentativa e erro, recebendo recompensas por acertos e penalidades por erros, aplicável em áreas como controle robótico e jogos.
#### Aplicações
Machine Learning pode ser aplicado em diversas áreas:

- Financeiro: Previsão de preços, detecção de fraudes, análise de risco e algoritmos de negociação.
- Saúde: Diagnóstico médico assistido, análise de imagem, descoberta de medicamentos e previsões de epidemias.
- Marketing: Segmentação de clientes, recomendações personalizadas, análise de sentimentos e previsão de churn.
- Indústria: Manutenção preditiva, otimização de processos e controle de qualidade.
- Tecnologia: Assistentes virtuais, reconhecimento de fala, visão computacional, e recomendações de conteúdo.
#### Principais informações
- Dados são fundamentais: ML depende de dados de qualidade para obter bons resultados.
- Algoritmos: Há várias técnicas, como árvores de decisão, redes neurais, regressão linear, máquinas de vetor de suporte (SVM), e mais.
- Avaliação do Modelo: É importante validar a precisão e ajustar os parâmetros do modelo.
- Infraestrutura: Ferramentas como Databricks, AWS, e bibliotecas como TensorFlow e Scikit-learn são comumente usadas.
Machine Learning permite que sistemas automatizem decisões complexas, oferecendo insights que podem transformar o funcionamento de empresas, instituições e até do cotidiano.
#### Conceitos
- Classe: é o que se buscar prever ou classificar
   - Ex: Espécie de planta, doença do paciente, se o cliente é bom pagador
   - Classe também é um atributo, é uma caracteristica.
   - Podemos ter atributos nominais ou numéricos (espécie de uma plata e comprimento).
   ##### Classificações
   - Classificação binária: é a classificação do objetivo em SIM ou NÃO.(Possui diferentes valores mas todas as clases estão na mesma coluna, mas possui apenas duas possibilidades).
   - Classificação multiclasse: é quanto temos mais de duas possibilidades(Possui diferentes valores mas todas as clases estão na mesma coluna).
   - Classificação multilabel: Se trata do conceito de classificar algo em mais de uma classe ao mesmo tempo. Ex: Uma música pode ser suave, clássica. E também pode ser uma música que não é moderna (Imaginamos que o que eu quero prever está em mais de uma coluna e uma linha pode ser classificada em mais de uma coluna ao mesmo tempo).
- Dimensão ou atributo: são as características usadas como parâmetros para classificar
- Instância: é uma observação
- Relação: conjunto de dados
#### Considerações sobre o modelo
- Depende do Algoritmo utilizado
- Pode perder a eficiência
- Muito específico do negócio
#### Regressão
- Aplicado quando a classe é contínua
- Exemplo: Você tem todas as informações de um carro e quer prever a potência de um motor(Será númerica)
#### Métrica de Erros
Quanto é necessario apurar um modelo que possui regressão(numeros reais ou inteiros) usamos outros tipos de métricas, agora não podemos mais avaliar se está certa ou errada, devemos avaliar a distancia da previsão feita com o valor que de fato ocorreu.
##### Root Mean Squared Error (RMSE)
Independente de Escala
 - O desvio padrão da amosta da diferença entre o previsto e o teste

|Previsto    |Realizado    |Dif. ao Quad.|
|:----------:|:-----------:|:-----------:|
|3,34        |3,00         |0,1156       |                   
|4,18        |4,00         |0,0324       |

- Pi = Valor previsto
- Ti = Valor realizado
- N = Quantidade de informações

RMSE =$$\sqrt{(∑^Ni=1(Pi-Ti)^2)/N}$$   

- RMSE =$$\sqrt{0,1483/2}$$   
- RMSE ≈ 0,2723



### 56. Machine Learning no Spark

#### Bibliotecas

 - spark.mllib
 - spark.ml

ML baseado em RDD está descontinuado, agora as implementações são todas em DataFrames.

#### Variáveis
Variáveis Independentes - São colunas distintas
Variável Dependente - Outra coluna, que normalmente é apenas uma coluna 

#### Variáveis No spark
Normalmente todas as váriaveis independentes devem compor uma mesma coluna. Cria-se um vetor único, que é adicionado em uma nova coluna no Dataframe

#### One HotEncoding
- Machine Learning suporta apenas números
- Atributos categóricos devem ser transformados em colunas, recebendo numeros para true ou false.

##### One HotEnconding no spark
- Existe o problema que se o atributo tiver muitos valores, serão criadas muitas colunas. Ex: Estados de um pais.
- Para isso o spark permite o uso de matriz esparsa. Imagine que tivemossemos 27 estados, para cada linhas teriamos 26 zeros e apenas 1 um, a matriz esparsa compacta as matrizes que contém muitos valores 0.

#### Formulas no R
**R** permite definir o modelo através de fórmula.
- Spark implemente Rformula
 - Aplica One HotEnconding e combina variáveis independentes em uma única coluna.

#### Pipelines
- Transformer: Transforma um DF em outro DF
- Estimator: Fir em DF para produzier um Transformer
- Pipeline: conecta Transformers Estimators para Produzir modelo
- Parâmetros: Transformers e Estimators compartilham uma Api para definir parâmetros

### 57. Preparando Dados para Regressão

In [2]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler


#criando dataframe
carros_tmp = spark.read.csv(r'C:\Users\viser\OneDrive\Documentos\Cursos\UDEMY_Formação Spark com pySpark\Material\files\Carros.csv', inferSchema=True, header=True, sep=';')
carros = carros_tmp.select('Consumo', 'Cilindros', 'Cilindradas', 'HP')
carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [3]:
# criar vetor de caracteristicas
# inputCols - Colunas que possuem as informações que serão avaliadas (Variaveis independentes)
# outputCol - Nome da coluna que vai receber as informações atraves de um vetor

vtcaracteristicas = VectorAssembler(inputCols=[('Consumo'), ('Cilindros'), ('Cilindradas')], outputCol='caracteristicas')

In [4]:
# aplicando vetorização nos dados 
carros = vtcaracteristicas.transform(carros)
carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [5]:
# dividir os dados para treino do modelo e dados para teste (divisão feita em porcentagem)
carrosTreino, CarrosTeste = carros.randomSplit([0.7,0.3])
print(carrosTreino.count())
print(CarrosTeste.count())

19
13


### 58. Criando um Modelo de Regressão Linear

In [6]:
# criando modelo do tipo de regressão com os dados de treino

# featuresCol - Coluna que foi vetorizadas com as informações (Variaveis independentes)
# labelCol - Coluna que possui as informações verdadeiras de histórico (Variavel dependente)

reglin = LinearRegression(featuresCol='caracteristicas', labelCol='HP')
modelo = reglin.fit(carrosTreino)

In [7]:
# Com o modelo criado, já podemos fazer a previsão - método transform do modelo
previsao = modelo.transform(CarrosTeste)
previsao.show(5)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     15|        8|        301|335|  [15.0,8.0,301.0]|192.79015322731442|
|    104|        8|        472|205| [104.0,8.0,472.0]|196.17423250681458|
|    133|        8|        350|245| [133.0,8.0,350.0]|197.62144504269622|
|    155|        8|        318|150| [155.0,8.0,318.0]|198.60194185479588|
|    164|        8|       2758|180|[164.0,8.0,2758.0]|194.24737111868768|
+-------+---------+-----------+---+------------------+------------------+
only showing top 5 rows



In [8]:
# Criando métrica de avaliação
# predictionCol - Coluna que tem as informações da previsão
# labelCol - Coluna que possui as informaçoes verdadeiras (é onde está a classe)
# metricName - é a métrica que gostariamos de utiliza para avaliar 

avaliar = RegressionEvaluator(predictionCol='prediction', labelCol='HP', metricName='rmse')

# avaliando o dataframe "previsao"
rmse = avaliar.evaluate(previsao)

# verificar o valor retornado, no caso do RMSE quando menor melhor, seria mais ou menos o tanto que ele errou.
print(rmse)

47.552490839548454


In [9]:
# criando motelo usando Random Forest Regressor

rfreg = RandomForestRegressor(featuresCol='caracteristicas', labelCol='HP')
modelo2 = rfreg.fit(carrosTreino)

In [10]:
previsao2 = modelo2.transform(CarrosTeste)
previsao2.show(5)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     15|        8|        301|335|  [15.0,8.0,301.0]|205.55337301587306|
|    104|        8|        472|205| [104.0,8.0,472.0]|194.26289682539684|
|    133|        8|        350|245| [133.0,8.0,350.0]|  228.623373015873|
|    155|        8|        318|150| [155.0,8.0,318.0]|205.81587301587305|
|    164|        8|       2758|180|[164.0,8.0,2758.0]| 180.0575396825397|
+-------+---------+-----------+---+------------------+------------------+
only showing top 5 rows



In [11]:
# comparando cada tecnica de modelo

rmse = avaliar.evaluate(previsao)
rmse2 = avaliar.evaluate(previsao2)

print(rmse)
print(rmse2)

47.552490839548454
44.98310569434564


### 59. Preparando Dados para Classificação (Classificação de churn binária)

In [12]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

churn = spark.read.csv(r"C:\Users\viser\OneDrive\Documentos\Cursos\UDEMY_Formação Spark com pySpark\Material\files\Churn.csv", inferSchema=True, header=True, sep=';')

In [13]:
#RFormula faz vários tipos de transformações

formula = RFormula(formula="Exited ~ .", featuresCol="features", labelCol="label", handleInvalid="skip")
churn_trans = formula.fit(churn).transform(churn).select('features', 'label')

churn_trans.show(truncate=False)

+----------------------------------------------------------------+-----+
|features                                                        |label|
+----------------------------------------------------------------+-----+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]        |1.0  |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]  |0.0  |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]  |1.0  |
|(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0])          |0.0  |
|[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0]   |0.0  |
|[645.0,0.0,0.0,1.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7]|1.0  |
|[822.0,1.0,0.0,1.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0]           |0.0  |
|[376.0,0.0,1.0,0.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7]|1.0  |
|[501.0,1.0,0.0,1.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0]   |0.0  |
|[684.0,1.0,0.0,1.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0]  |0.0  |
|[528.0,1.0,0.0,1.0,31.0,6.0,1.0201672E7,2.0,0.0,0.

### 60. Criando um Modelo de Classificação

In [14]:
churnTreino, churnTeste = churn_trans.randomSplit([0.7,0.3])
print(churnTreino.count())
print(churnTeste.count())

7067
2933


In [15]:
dt = DecisionTreeClassifier(labelCol='label', featuresCol='features')
modelo = dt.fit(churnTreino)

In [16]:
previsao_class = modelo.transform(churnTeste)
previsao_class.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(11,[0,1,4,5,7,10...|  1.0|  [34.0,240.0]|[0.12408759124087...|       1.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  1.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4399.0,495.0]|[0.89885574172456...|       0.0|
|(11,[0,1,4,

In [17]:
avaliar = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
areaUnderROC = avaliar.evaluate(previsao_class)

print(areaUnderROC)

0.7124250714372676


### 61. Pipelines

In [18]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

carros_tmp = spark.read.csv(r"C:\Users\viser\OneDrive\Documentos\Cursos\UDEMY_Formação Spark com pySpark\Material\files\Carros.csv", inferSchema=True, header=True, sep=';')
carros_tmp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [19]:
carros = carros_tmp.select("Consumo","Cilindros","Cilindradas","HP")
carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [20]:
#dividifrom r em treino e teste
carrosTreino, carrosTeste = carros.randomSplit([0.7,0.3])

In [21]:
#criar vetor de caracteristicas
veccaracteristicas = VectorAssembler(inputCols=[("Consumo"),("Cilindros"),("Cilindradas")],outputCol="caracteristicas")

In [22]:
#aplicamos em dados de treino
vec_carrosTreino = veccaracteristicas.transform(carrosTreino)
vec_carrosTreino.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     15|        8|        301|335| [15.0,8.0,301.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    104|        8|        472|205|[104.0,8.0,472.0]|
|    133|        8|        350|245|[133.0,8.0,350.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [23]:
#criamos modelo 
#coluna caracteristicas foi criaada pelo VectorAssembler
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")
modelo = reglin.fit(vec_carrosTreino)

In [ ]:
#pipelines permite criar um fluxo do processo
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[veccaracteristicas,reglin])
#fit 
pipelineModel = pipeline.fit(carros)
#previsão
previsao = pipelineModel.transform(carros)
previsao.show(5)


+-------+---------+-----------+---+-----------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|162.32154816816646|
|     21|        6|        160|110| [21.0,6.0,160.0]|162.32154816816646|
|    228|        4|        108| 93|[228.0,4.0,108.0]| 82.51715587712931|
|    214|        6|        258|110|[214.0,6.0,258.0]|141.86680518718754|
|    187|        8|        360|175|[187.0,8.0,360.0]|202.93528239714834|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



### 61. Ativididades: Faça você mesmo

In [40]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

iris = spark.read.csv(r"C:\Users\viser\OneDrive\Documentos\Cursos\UDEMY_Formação Spark com pySpark\Material\files\iris.csv", inferSchema=True, header=True)
iris.show(5)

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



In [41]:
#RFormula faz vários tipos de transformações

formula = RFormula(formula="class ~ .", featuresCol="features", labelCol="label", handleInvalid="skip")
iris = formula.fit(iris).transform(iris).select('features', 'label')

iris.show(truncate=False)

+-----------------+-----+
|features         |label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|0.0  |
|[4.9,3.0,1.4,0.2]|0.0  |
|[4.7,3.2,1.3,0.2]|0.0  |
|[4.6,3.1,1.5,0.2]|0.0  |
|[5.0,3.6,1.4,0.2]|0.0  |
|[5.4,3.9,1.7,0.4]|0.0  |
|[4.6,3.4,1.4,0.3]|0.0  |
|[5.0,3.4,1.5,0.2]|0.0  |
|[4.4,2.9,1.4,0.2]|0.0  |
|[4.9,3.1,1.5,0.1]|0.0  |
|[5.4,3.7,1.5,0.2]|0.0  |
|[4.8,3.4,1.6,0.2]|0.0  |
|[4.8,3.0,1.4,0.1]|0.0  |
|[4.3,3.0,1.1,0.1]|0.0  |
|[5.8,4.0,1.2,0.2]|0.0  |
|[5.7,4.4,1.5,0.4]|0.0  |
|[5.4,3.9,1.3,0.4]|0.0  |
|[5.1,3.5,1.4,0.3]|0.0  |
|[5.7,3.8,1.7,0.3]|0.0  |
|[5.1,3.8,1.5,0.3]|0.0  |
+-----------------+-----+
only showing top 20 rows



In [42]:
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])
print(irisTreino.count())
print(irisTeste.count())

108
42


In [ ]:
nb = NaiveBayes(smoothing=2, labelCol='label', featuresCol='features')
modelo = dt.fit(irisTreino)

In [44]:
previsao_iris = modelo.transform(irisTeste)
previsao_iris.show(5)

+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[4.4,3.2,1.3,0.2]|  0.0|[-10.864765395492...|[0.72573874227767...|       0.0|
|[4.6,3.1,1.5,0.2]|  0.0|[-11.283272490601...|[0.69127358832488...|       0.0|
|[4.8,3.0,1.4,0.1]|  0.0|[-10.764991932204...|[0.72969422887009...|       0.0|
|[4.8,3.4,1.9,0.2]|  0.0|[-12.522385166056...|[0.67360448122995...|       0.0|
|[4.9,2.4,3.3,1.0]|  1.0|[-17.074385273706...|[0.12450791427337...|       1.0|
+-----------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
avaliar_iris = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
avaliar_iris = avaliar_iris.evaluate(previsao_iris)

## esse modelo mostra 92 de certo
print(avaliar_iris)

0.9285714285714286


### 62. Solução Faça você mesmo